# HW2

In [58]:
# import libiaries
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import pandas as pd
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /Users/zazhu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Read Yelp review data

In [28]:
review = pd.read_json('yelp_academic_dataset_review.json',lines=True)

In [48]:
review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


## Functions

In [136]:
def preprocess(df):
    '''tokenization, normalization (e.g. convert to lowercase, remove non-alphanumeric chars, numbers, etc.）'''
    text_norm = []
    text_norm_string = []
    for i in range(0,len(df)): 
        row = df.iloc[[i]]
        strings = row.to_string(header = False)       
        # tokenize 
        token = word_tokenize(strings)
        # only keep alphabet and get lower case
        alpha = [word.lower() for word in token if word.isalpha()]
        # nomolized taxt list 
        text_norm.append(alpha)
        text_norm_string.append(" ".join(alpha))
    # Return to text file
    with open('yelp_text_norm.txt', 'w') as f:
        for l in text_norm:
            f.write("%s\n" % l)
    
    return(text_norm)

# find top k closest neighbors of target words under given models 
def find_neighbor(model, words, k):
    dic = {}
    for word in words:
        top5 = model.wv.most_similar(word)[:k]
        dic[word] = top5
    
    model_df = pd.DataFrame(dic)
    
    return model_df  

## Preprocess

In [88]:
# data is too large, choose only 100k records of data
nomalized_text = preprocess(review.head(100000))

In [145]:
nomalized_text[0][:10]

['as',
 'someone',
 'who',
 'has',
 'worked',
 'with',
 'many',
 'museums',
 'i',
 'was']

In [125]:
#word2vec models with varying parameters
model_cbow1 = gensim.models.Word2Vec(nomalized_text, window=5, workers=5, sg=0)
model_cbow2 = gensim.models.Word2Vec(nomalized_text, window=50, workers=5, sg=0)
model_sg = gensim.models.Word2Vec(nomalized_text, window=50, workers=5, sg=1)

In [143]:
# find top k=5 closest neighbors of 10 target words under given models 
words_10 = ["expensive", "summer", "hotpot", "husband","hotel", "salad", "chicago", "coffee", "gay","wonderful"]

model1_similar = find_neighbor(model_cbow1, words_10, k=5)
model2_similar = find_neighbor(model_cbow2, words_10, k=5)
model3_similar = find_neighbor(model_sg, words_10, k=5)

In [142]:
model1_similar

,expensive,summer,hotpot,husband,hotel,salad,chicago,coffee,gay,wonderful
0,"(pricey, 0.781383216381073)","(winter, 0.8724745512008667)","(szechuan, 0.7406690120697021)","(wife, 0.9544609785079956)","(casino, 0.7690605521202087)","(hummus, 0.6663291454315186)","(nyc, 0.8111834526062012)","(brew, 0.7164369225502014)","(biker, 0.7073823809623718)","(fantastic, 0.8936104774475098)"
1,"(overpriced, 0.7321226596832275)","(season, 0.696569561958313)","(mee, 0.7343357801437378)","(boyfriend, 0.9537715911865234)","(flamingo, 0.7388657927513123)","(falafel, 0.6496670842170715)","(hawaii, 0.7611942291259766)","(tea, 0.7086067199707031)","(hipsters, 0.6967926025390625)","(fabulous, 0.8719648122787476)"
2,"(pricy, 0.7063122987747192)","(evenings, 0.6701287627220154)","(manchurian, 0.7278831005096436)","(bf, 0.9168609976768494)","(trump, 0.7159742712974548)","(sandwich, 0.6488436460494995)","(seattle, 0.7306854128837585)","(latte, 0.7080820798873901)","(folk, 0.6902546882629395)","(terrific, 0.8237217664718628)"
3,"(steep, 0.6992413997650146)","(summers, 0.6505776643753052)","(dumpling, 0.7258517742156982)","(fiancé, 0.903343677520752)","(motel, 0.7045632004737854)","(kale, 0.6405284404754639)","(boston, 0.7283211946487427)","(cappuccino, 0.6976271271705627)","(bearded, 0.6836799383163452)","(phenomenal, 0.8206644058227539)"
4,"(cheap, 0.6470549702644348)","(weather, 0.6453604698181152)","(dolsot, 0.7193117141723633)","(girlfriend, 0.9020626544952393)","(resort, 0.7035679221153259)","(spinach, 0.6389374136924744)","(cali, 0.7216947078704834)","(espresso, 0.6731268167495728)","(frat, 0.6739295721054077)","(superb, 0.8086722493171692)"


In [144]:
model2_similar

,expensive,summer,hotpot,husband,hotel,salad,chicago,coffee,gay,wonderful
0,"(pricey, 0.6952802538871765)","(winter, 0.8230222463607788)","(bim, 0.6086511611938477)","(boyfriend, 0.8037543296813965)","(resort, 0.7909456491470337)","(dressing, 0.8220903873443604)","(nyc, 0.6978552937507629)","(latte, 0.7728216648101807)","(females, 0.632547914981842)","(fabulous, 0.8343876600265503)"
1,"(overpriced, 0.6475316286087036)","(weather, 0.7351474761962891)","(sichuan, 0.6026174426078796)","(hubby, 0.7814106941223145)","(trump, 0.780063807964325)","(croutons, 0.7623512148857117)","(ohio, 0.6540114879608154)","(cappuccino, 0.7721251249313354)","(carey, 0.6119317412376404)","(fantastic, 0.8309641480445862)"
2,"(steep, 0.6104059219360352)","(sun, 0.6641788482666016)","(congee, 0.5904984474182129)","(fiancé, 0.7773969173431396)","(hotels, 0.7790665030479431)","(vinaigrette, 0.7429793477058411)","(brooklyn, 0.651384711265564)","(espresso, 0.7705351114273071)","(booty, 0.6030271053314209)","(phenomenal, 0.7880927324295044)"
3,"(outrageous, 0.5708798766136169)","(evenings, 0.6599687337875366)","(gai, 0.5903284549713135)","(fiance, 0.736877977848053)","(hilton, 0.7637650370597839)","(romaine, 0.7353838682174683)","(coast, 0.6509166359901428)","(mocha, 0.7658597230911255)","(costumes, 0.6024737358093262)","(incredible, 0.7764356136322021)"
4,"(paying, 0.5707785487174988)","(season, 0.6503993272781372)","(bi, 0.586243212223053)","(son, 0.7368299961090088)","(staying, 0.742067813873291)","(kale, 0.7155261039733887)","(pittsburgh, 0.644057035446167)","(coffees, 0.7435194253921509)","(songs, 0.6015019416809082)","(outstanding, 0.7319576740264893)"


In [117]:
model3_similar

,expensive,summer,hotpot,husband,hotel,salad,chicago,coffee,gay,wonderful
0,"(pricey, 0.8402361869812012)","(winter, 0.8067606687545776)","(steamed, 0.733089804649353)","(wife, 0.8671326041221619)","(hotels, 0.9116913080215454)","(dressing, 0.9131932258605957)","(giordano, 0.7606273889541626)","(cappuccino, 0.8280515074729919)","(dance, 0.7103992104530334)","(fantastic, 0.9032050371170044)"
1,"(considering, 0.8173071146011353)","(weather, 0.7725754976272583)","(dishes, 0.7183525562286377)","(hubby, 0.8439088463783264)","(stay, 0.9033572673797607)","(vinaigrette, 0.8448192477226257)","(coast, 0.7211610078811646)","(latte, 0.8268523812294006)","(shirtless, 0.708240270614624)","(fabulous, 0.8861724138259888)"
2,"(price, 0.8122135400772095)","(sun, 0.7136780023574829)","(dumplings, 0.7179104685783386)","(boyfriend, 0.8380752801895142)","(staying, 0.8926018476486206)","(croutons, 0.8365826606750488)","(giordanos, 0.7184209823608398)","(lattes, 0.8261185884475708)","(dancers, 0.7041320204734802)","(amazing, 0.8594306707382202)"
3,"(than, 0.8112142086029053)","(outdoor, 0.6982121467590332)","(noodle, 0.7119240760803223)","(my, 0.8033021688461304)","(rooms, 0.8736310005187988)","(lettuce, 0.8355463743209839)","(pizzerias, 0.7082206606864929)","(coffees, 0.817154586315155)","(tunes, 0.7004091739654541)","(phenomenal, 0.8493140339851379)"
4,"(overpriced, 0.8081164956092834)","(during, 0.6711623668670654)","(tofu, 0.7048190832138062)","(had, 0.800942063331604)","(resort, 0.866202175617218)","(chopped, 0.8350917100906372)","(pittsburgh, 0.7080279588699341)","(espresso, 0.8098669052124023)","(gaga, 0.6951059103012085)","(incredible, 0.8466864228248596)"
